In [1]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
import serial
import wave
from scipy.signal import butter, filtfilt

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024*4  # samples per frame
FORMAT = pyaudio.paInt16  # audio format (bytes per sample)
CHANNELS = 1  # single channel for microphone
RATE = 44100  # samples per second

wav_file = wave.open(r"C:\Users\rajka\Downloads\My Song 7.wav", 'rb')

# Set up serial communication with Arduino
arduino = serial.Serial('COM4', 9600)
time.sleep(2)  # Wait for Arduino to initialize

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=p.get_format_from_width(wav_file.getsampwidth()),
    channels=wav_file.getnchannels(),
    rate=wav_file.getframerate(),
    input=False,
    output=True,
    frames_per_buffer=CHUNK
)

# create matplotlib figure and axes
fig, ax2 = plt.subplots(figsize=(15, 7))

# Define the frequency bins for the bar plot
# freq_bins = [40,50,60,100,150,200,250,300,400,450,600,800,1000]
freq_bins=[50,300,500]

# Create the x-axis labels for the bar plot
labels = [str(freq) for freq in freq_bins]

# Create the bar plot
bar_fft = ax2.bar(labels[:-1], np.zeros(len(labels) - 1))

# format spectrum axes
ax2.set_ylim(0, 1)

print('stream started')
chosen_frequencies = [50,300]

# for measuring frame rate
frame_count = 0
start_time = time.time()

# Read the audio data from the .wav file
data = wav_file.readframes(CHUNK)

while len(data) > 0:

    # convert data to integers, make np array, then offset it by 128
    data_int = np.array(struct.unpack(f"{2 * CHUNK}h", data), dtype=np.int16) + 128

    # compute FFT
    yf = fft(data_int)

    # Update the bar heights with the frequency values
    # for i in range(len(freq_bins) -1):
    #     # start_index = np.abs(np.fft.fftfreq(CHUNK, 1/RATE) - freq_bins[i]).argmin()
    #     # end_index = np.abs(np.fft.fftfreq(CHUNK, 1/RATE) - freq_bins[i + 1]).argmin()
    #     index = np.abs(np.fft.fftfreq(CHUNK, 1/RATE) - freq).argmin()
    #     value = np.abs(yf[index]) / (CHUNK*128)
    #     bar_fft[i].set_height(value / (CHUNK*128))
    # Update the bar heights with the frequency values
    for i in range(len(freq_bins) - 1):
        start_index = np.abs(np.fft.fftfreq(CHUNK, 1 / RATE) - freq_bins[i]).argmin()
        end_index = np.abs(np.fft.fftfreq(CHUNK, 1 / RATE) - freq_bins[i + 1]).argmin()
        bar_fft[i].set_height(np.abs(yf[start_index:end_index]).mean() / (CHUNK * 128))


    stream.write(data)

    # Send maximum frequencies and values to Arduino
    for freq in chosen_frequencies:
        index = np.abs(np.fft.fftfreq(CHUNK, 1/RATE) - freq).argmin()
        value = np.abs(yf[index]) / (CHUNK*128)
        print(f"Frequency: {freq} Hz, Value: {value}")
        message = f"{freq}:{value}\n".encode()  # Encode message as bytes
        arduino.write(message)

    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1

    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)

        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

    # Read the next chunk of audio data from the .wav file
    data = wav_file.readframes(CHUNK)

print('Playback finished')


stream started
Frequency: 50 Hz, Value: 93.9944791958704
Frequency: 300 Hz, Value: 4.5168360439544335
Frequency: 50 Hz, Value: 48.69796313984328
Frequency: 300 Hz, Value: 0.5588605378066458
Frequency: 50 Hz, Value: 13.823457434649518
Frequency: 300 Hz, Value: 0.23694053170845333
Frequency: 50 Hz, Value: 4.255466285115078
Frequency: 300 Hz, Value: 0.11935266198190868
Frequency: 50 Hz, Value: 1.2614682227638563
Frequency: 300 Hz, Value: 0.04028052998407135
Frequency: 50 Hz, Value: 0.2752758367737142
Frequency: 300 Hz, Value: 0.00819387020446595
Frequency: 50 Hz, Value: 0.63064947728024
Frequency: 300 Hz, Value: 10.489149302492853
Frequency: 50 Hz, Value: 0.05279987947303432
Frequency: 300 Hz, Value: 0.0009675638491306184
Frequency: 50 Hz, Value: 0.0017282808872078146
Frequency: 300 Hz, Value: 0.0017612961515894314
Frequency: 50 Hz, Value: 0.0036311039875618445
Frequency: 300 Hz, Value: 0.0006884064375912199
Frequency: 50 Hz, Value: 0.002851082019380509
Frequency: 300 Hz, Value: 0.0006642

error: unpack requires a buffer of 16384 bytes